In [1]:
import re
import numpy as np

In [2]:
def countcapital(word):
    count = 0
    for i in word:
        if i.isupper():
            count += 1
    return count
countcapital("HEere")

2

In [3]:
def checknum(word):
    
    pattern = re.compile('[0-9]+')
    result = pattern.findall(word)
#     print(result)
    return len(result)
    
# print(checknum('1jhkj42hjh'))


def getfeatures(word):
    # 定义向量features
    features = []
    
    # 判断是否全为字母
    features.append(word.isalpha()+0)
#     print("是否全为字母\t",word.isalpha()+0)
    
    # 判断字母是否全是大写
    features.append(word.isupper()+0)
#     print("是否所有字母都是大写\t",word.isupper()+0)
    
    # 判断是否全部是数字
    features.append(word.isdigit()+0)
#     print("是否全部是数字\t",word.isdigit()+0)
    
    #判断有多少数字
    features.append(checknum(word)+0)
#     print("有多少数字\t",checknum(word)+0)
    
    # 判断是否全部为字母或数字组成
    features.append(word.isalnum()+0)
#     print("是否全为字母或者数字\t",word.isalnum()+0)
    
    # 判断首字母是否大写
    features.append(word.istitle()+0)
#     print("首字母是否大写\t",word.istitle()+0)
    # 返回结果
#     return np.array(features)

#    大写字母个数
    c = countcapital(word)
    features.append(c)
#   单词长度
    l = len(word)
    features.append(l)
#    以n结尾
#    字母占比
    
#    数字占比
    features.append(checknum(word)/l+0)
#    大写字母占比
    features.append(c/l)

    return features
getfeatures('djflskjafiLLLIIJ12434')
# checknum('122l22')

[0, 0, 0, 1, 1, 0, 6, 21, 0.047619047619047616, 0.2857142857142857]

In [4]:
def getag(tag):
    tag = tag.replace("\n","")
    if tag == "TAG":
        return 0
    if tag =="GENE1":
        return 1
    if tag == "GENE2":
        return 2

In [5]:
def getvec(features):
    l = len(features[0])
    start = [[0]*l]
    end = [[0]*l]
    result = []
    
    
    features = start + features + end
#     start.extend(features)
#     print(start)
#     start.extend(end)
#     print(start)
    
#     print(features)
    
    for i in range(1,len(features)-1):
        feat = features[i-1] + features[i] + features[i+1]
        feat = np.array(feat)
        result.append(np.hstack( (feat,feat ** 2) ))
    
    return result

getvec([[1,2,2,2,2,23,3,3,3,3,3],[3,4,4,4,4,4],[5,6,6,6,6,6]])
        
    

[array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   2,
          2,   2,   2,  23,   3,   3,   3,   3,   3,   3,   4,   4,   4,
          4,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          1,   4,   4,   4,   4, 529,   9,   9,   9,   9,   9,   9,  16,
         16,  16,  16,  16]),
 array([  1,   2,   2,   2,   2,  23,   3,   3,   3,   3,   3,   3,   4,
          4,   4,   4,   4,   5,   6,   6,   6,   6,   6,   1,   4,   4,
          4,   4, 529,   9,   9,   9,   9,   9,   9,  16,  16,  16,  16,
         16,  25,  36,  36,  36,  36,  36]),
 array([ 3,  4,  4,  4,  4,  4,  5,  6,  6,  6,  6,  6,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  9, 16, 16, 16, 16, 16, 25, 36, 36, 36, 36,
        36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])]

In [6]:
f = open('data/train.tag', 'r')

X = []
Y = []

for line in f.readlines():

    # 若该行包含“_”，则证明该行不是序号，是数据
    if "_" in line:
        # print(line)

        # 将每一行按照空格分隔，则会返回一个列表，列表内的元素为 单词_标签 的形式
        line = line.split(" ")
#         print(line)

        words = []
        tags = []
        for word_tag in line:
            word_tag = word_tag.split("_")
            word = word_tag[0]
            tag = word_tag[1]
            
            features = getfeatures(word)
            
            
            # 后面可以加一些对word和tag的操作
            words.append(features)
            tags.append(getag(tag))
#         print(getvec(words))
#         print(tags)
        
        
        X += getvec(words)
        Y += tags
        
        

            

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.1)
# print(x_train)
# print(x_test)
# print(y_train)
# print(y_test)

In [8]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2',class_weight='balanced',C=10.0,verbose=True)
model.fit(x_train,y_train)
train_score = model.score(x_train,y_train)
test_score = model.score(x_test,y_test)
print('train score: {train_score:.6f}; test_score:{test_score:.6f}'
      .format(train_score=train_score,
             test_score=test_score))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]train score: 0.872457; test_score:0.870722


In [9]:
y_pred_proba = model.predict_proba(x_test)
i=0
while i < 10000:
    
    if y_test[i] != 0:
        print(x_test[i],y_pred_proba[i],y_test[i])
    i += 1

[  1.   0.   0.   0.   1.   0.   0.  13.   0.   0.   1.   1.   0.   0.
   1.   0.   3.   3.   0.   1.   1.   0.   0.   0.   1.   0.   0.  11.
   0.   0.   1.   0.   0.   0.   1.   0.   0. 169.   0.   0.   1.   1.
   0.   0.   1.   0.   9.   9.   0.   1.   1.   0.   0.   0.   1.   0.
   0. 121.   0.   0.] [0.20715828 0.39616906 0.39667266] 2
[ 1.          0.          0.          0.          1.          0.
  0.          4.          0.          0.          1.          0.
  0.          0.          1.          0.          0.          5.
  0.          0.          0.          0.          0.          1.
  1.          0.          0.          3.          0.33333333  0.
  1.          0.          0.          0.          1.          0.
  0.         16.          0.          0.          1.          0.
  0.          0.          1.          0.          0.         25.
  0.          0.          0.          0.          0.          1.
  1.          0.          0.          9.          0.11111111  0.        

  1.  0.  0. 49.  0.  0.] [0.60981667 0.199977   0.19020633] 1
[ 1.          0.          0.          0.          1.          1.
  1.          3.          0.          0.33333333  0.          0.
  0.          0.          0.          0.          0.          1.
  0.          0.          1.          0.          0.          0.
  1.          1.          1.          5.          0.          0.2
  1.          0.          0.          0.          1.          1.
  1.          9.          0.          0.11111111  0.          0.
  0.          0.          0.          0.          0.          1.
  0.          0.          1.          0.          0.          0.
  1.          1.          1.         25.          0.          0.04      ] [0.65109365 0.180007   0.16889935] 2
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0.  8.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  0. 64.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0. 

  1.  0.  0. 81.  0.  0.] [0.19332946 0.41191891 0.39475163] 1
[  1.   0.   0.   0.   1.   0.   0.   9.   0.   0.   1.   0.   0.   0.
   1.   0.   0.  13.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.
   0.   0.   1.   0.   0.   0.   1.   0.   0.  81.   0.   0.   1.   0.
   0.   0.   1.   0.   0. 169.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   1.   0.   0.] [0.58302547 0.19724765 0.21972688] 1
[ 1.      0.      0.      0.      1.      0.      0.      3.      0.
  0.      1.      0.      0.      0.      1.      0.      1.      4.
  0.      0.25    1.      0.      0.      0.      1.      0.      0.
  6.      0.      0.      1.      0.      0.      0.      1.      0.
  0.      9.      0.      0.      1.      0.      0.      0.      1.
  0.      1.     16.      0.      0.0625  1.      0.      0.      0.
  1.      0.      0.     36.      0.      0.    ] [0.47865638 0.2655746  0.25576902] 2
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  1.  0.  3.  3.
  0.  1.  1.  0.  0.

 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0.18926145 0.41192169 0.39881686] 2
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 3. 3. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 9. 9.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0.15420814 0.42828559 0.41750627] 1
[ 1.  0.  0.  0.  1.  0.  0.  2.  0.  0.  1.  0.  0.  0.  1.  0.  0.  7.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  3.  0.  0.  1.  0.  0.  0.  1.  0.
  0.  4.  0.  0.  1.  0.  0.  0.  1.  0.  0. 49.  0.  0.  1.  0.  0.  0.
  1.  0.  0.  9.  0.  0.] [0.77799705 0.11231097 0.10969199] 2
[1. 1. 0. 0. 1. 0. 2. 2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1.
 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 4. 4. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0.] [0.12522649 0.43769976 0.43707375] 2
[0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   1.   0.   1.
 1.   1.   1.   2.   0.5  0.5  0.   0.   0.   0.   0.   0.   0.   1.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.

  1.  0.  0. 36.  0.  0.] [0.75121183 0.12469866 0.12408951] 2
[0.00e+00 1.00e+00 0.00e+00 1.00e+00 1.00e+00 1.00e+00 1.00e+00 4.00e+00
 2.50e-01 2.50e-01 1.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00
 0.00e+00 1.30e+01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00
 0.00e+00 1.00e+00 1.00e+00 1.00e+00 1.00e+00 1.60e+01 6.25e-02 6.25e-02
 1.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00 0.00e+00 1.69e+02
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 1.00e+00 0.00e+00 0.00e+00] [0.37640111 0.28985061 0.33374827] 1
[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 3.00000000e+00
 0.00000000e+00 3.33333333e-01 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 5.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0.23559503 0.37797604 0.38642893] 1
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 3. 3. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 9. 9.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0.15420814 0.42828559 0.41750627] 2
[  1.   0.   0.   0.   1.   0.   0.  11.   0.   0.   1.   1.   0.   0.
   1.   0.   2.   2.   0.   1.   0.   0.   0.   0.   0.   0.   0.   1.
   0.   0.   1.   0.   0.   0.   1.   0.   0. 121.   0.   0.   1.   1.
   0.   0.   1.   0.   4.   4.   0.   1.   0.   0.   0.   0.   0.   0.
   0.   1.   0.   0.] [0.16488336 0.41903665 0.41607999] 1
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  1.  1.  0.  0.  1.
  1.  0.  1.  0.  0.  0.  1.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  1.  1.  1.  0.  0.  1.  1.  0.  1.  0.  0. 

  0.  0.  0.  1.  0.  0.] [0.77700723 0.11078738 0.11220539] 1
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 3. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 9.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0.7539452  0.1207518  0.12530301] 2
[ 1.  0.  0.  0.  1.  0.  0.  3.  0.  0.  1.  0.  0.  0.  1.  0.  0.  8.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  9.  0.  0.  1.  0.  0.  0.  1.  0.
  0.  9.  0.  0.  1.  0.  0.  0.  1.  0.  0. 64.  0.  0.  1.  0.  0.  0.
  1.  0.  0. 81.  0.  0.] [0.61014992 0.20013623 0.18971385] 1
[1.00e+00 0.00e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00 0.00e+00 3.00e+00
 0.00e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00 1.00e+00 1.00e+00 0.00e+00
 2.00e+00 4.00e+00 2.50e-01 5.00e-01 1.00e+00 0.00e+00 0.00e+00 0.00e+00
 1.00e+00 0.00e+00 0.00e+00 8.00e+00 0.00e+00 0.00e+00 1.00e+00 0.00e+00
 0.00e+00 0.00e+00 1.00e+00 0.00e+00 0.00e+00 9.00e+00 0.00e+00 0.00e+00
 0.00e+00 1.00e+00 0.00e+00 1.00e+00 1.00e+00 0.00e+00 4.00e+00 1.60e

  1.  0.  0. 36.  0.  0.] [0.62743234 0.19043249 0.18213517] 1
[ 1.  0.  0.  0.  1.  0.  0.  6.  0.  0.  1.  0.  0.  0.  1.  0.  0.  7.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  7.  0.  0.  1.  0.  0.  0.  1.  0.
  0. 36.  0.  0.  1.  0.  0.  0.  1.  0.  0. 49.  0.  0.  1.  0.  0.  0.
  1.  0.  0. 49.  0.  0.] [0.63761188 0.1872175  0.17517063] 2
[ 1.  0.  0.  0.  1.  0.  0.  8.  0.  0.  1.  0.  0.  0.  1.  0.  0.  2.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  5.  0.  0.  1.  0.  0.  0.  1.  0.
  0. 64.  0.  0.  1.  0.  0.  0.  1.  0.  0.  4.  0.  0.  1.  0.  0.  0.
  1.  0.  0. 25.  0.  0.] [0.87009577 0.06272356 0.06718066] 1
[ 1.    0.    0.    0.    1.    0.    0.    4.    0.    0.    1.    0.
  0.    0.    1.    1.    1.    2.    0.    0.5   1.    0.    0.    0.
  1.    0.    0.    3.    0.    0.    1.    0.    0.    0.    1.    0.
  0.   16.    0.    0.    1.    0.    0.    0.    1.    1.    1.    4.
  0.    0.25  1.    0.    0.    0.    1.    0.    0.    9.    0.    0.  ] [0.57351285 0.2

  0.  0.  0.  1.  0.  0.] [0.57473814 0.21757198 0.20768988] 1
[ 1.  0.  0.  0.  1.  0.  0.  2.  0.  0.  1.  0.  0.  0.  1.  0.  0.  4.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  5.  0.  0.  1.  0.  0.  0.  1.  0.
  0.  4.  0.  0.  1.  0.  0.  0.  1.  0.  0. 16.  0.  0.  1.  0.  0.  0.
  1.  0.  0. 25.  0.  0.] [0.8111579  0.09463349 0.09420861] 2
[1.         0.         0.         0.         1.         1.
 1.         3.         0.         0.33333333 0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.         0.         1.         1.
 1.         0.         0.         1.         1.         0.
 1.         0.         0.         0.         1.         1.
 1.         9.         0.         0.11111111 0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.         0.         1.         1.
 1.         0.         0.         1.         1.         0.        ] [0.32880189 0.32332028 0.34787783] 2
[ 1.  1.  0.  0.  1

In [10]:
y_pred = model.predict(x_test)

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("accuracy score:",accuracy_score(y_test,y_pred ))
print("precision_score",precision_score(y_test,y_pred, average='macro'))
print("recall_score:",recall_score(y_test,y_pred, average='macro'))
print("f1_score:",f1_score(y_test,y_pred, average='macro'))

print("precision_score",precision_score(y_test,y_pred, average='micro'))
print("recall_score:",recall_score(y_test,y_pred, average='micro'))
print("f1_score:",f1_score(y_test,y_pred, average='micro'))

print("precision_score",precision_score(y_test,y_pred, average='weighted'))
print("recall_score:",recall_score(y_test,y_pred, average='weighted'))
print("f1_score:",f1_score(y_test,y_pred, average='weighted'))

print("precision_score",precision_score(y_test,y_pred, average=None))
print("recall_score:",recall_score(y_test,y_pred, average=None))
print("f1_score:",f1_score(y_test,y_pred, average=None))

accuracy score: 0.8707215071663487
precision_score 0.4545226286781176
recall_score: 0.44277989011948127
f1_score: 0.4457260468359698
precision_score 0.8707215071663487
recall_score: 0.8707215071663487
f1_score: 0.8707215071663488
precision_score 0.8605705363509792
recall_score: 0.8707215071663487
f1_score: 0.8651452020737224
precision_score [0.93232314 0.22758621 0.20365854]
recall_score: [0.94658245 0.25169492 0.13006231]
f1_score: [0.93939869 0.23903421 0.15874525]


In [12]:
y_pred = model.predict(x_train)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("accuracy score:",accuracy_score(y_train,y_pred ))
print("precision_score",precision_score(y_train,y_pred, average='macro'))
print("recall_score:",recall_score(y_train,y_pred, average='macro'))
print("f1_score:",f1_score(y_train,y_pred, average='macro'))

# print("precision_score",precision_score(y_train,y_pred, average='micro'))
# print("recall_score:",recall_score(y_train,y_pred, average='micro'))
# print("f1_score:",f1_score(y_train,y_pred, average='micro'))

# print("precision_score",precision_score(y_train,y_pred, average='weighted'))
# print("recall_score:",recall_score(y_train,y_pred, average='weighted'))
# print("f1_score:",f1_score(y_train,y_pred, average='weighted'))

print("precision_score",precision_score(y_train,y_pred, average=None))
print("recall_score:",recall_score(y_train,y_pred, average=None))
print("f1_score:",f1_score(y_train,y_pred, average=None))

accuracy score: 0.8724572428820846
precision_score 0.4592134930689937
recall_score: 0.4520132993673915
f1_score: 0.4537505152856028
precision_score [0.93505086 0.25228091 0.19030871]
recall_score: [0.94529288 0.2772004  0.13354662]
f1_score: [0.94014397 0.26415425 0.15695332]
